# 데이터셋 클렌징
- `recipe.json` 원본 파일 -> 데이터 클렌징한 `recipe_cleaned.csv` 생성

- 필요한 파일은 `Confluence`에 `recipe.json`과 `normalized_pattern.pkl` 첨부되어 있습니다.
    ```
    datasets/
    ├── ingredients_cleand.csv (저장할 데이터셋, 클렌징한 데이터)
    ├── normalized_pattern.pkl (정규화 식 모음)
    └── recipe.json (원본 데이터셋)
    ```

## 데이터셋 폴더 생성
```bash
cd NangPaGo/NangPaGo-data
mkdir datasets
```
datasets 폴더를 생성 후, recipe.json을 넣기

## 가상환경 세팅
```bash
python3.10 -m venv py310
pip install -r requirements.txt
```


## RAW 데이터 전처리
- ingredients - Null 값인 경우 날림
- 데이터 타입 변경 (Object -> Numeric)

In [179]:
import json
import pandas as pd
import numpy as np
import re
import pickle

In [180]:
with open("datasets/recipe.json", "r", encoding="utf-8") as fname:
    js = json.load(fname)

In [181]:
df_raw = pd.DataFrame(js)
sorted_columns = [
    "RCP_SEQ", "RCP_NM", "RCP_WAY2", "RCP_PAT2", "INFO_WGT", "INFO_ENG", 
    "INFO_CAR", "INFO_PRO", "INFO_FAT", "INFO_NA", "HASH_TAG", 
    "ATT_FILE_NO_MAIN", "ATT_FILE_NO_MK", "RCP_PARTS_DTLS", 
    "MANUAL01", "MANUAL_IMG01", "MANUAL02", "MANUAL_IMG02", "MANUAL03", 
    "MANUAL_IMG03", "MANUAL04", "MANUAL_IMG04", "MANUAL05", "MANUAL_IMG05", 
    "MANUAL06", "MANUAL_IMG06", "MANUAL07", "MANUAL_IMG07", "MANUAL08", 
    "MANUAL_IMG08", "MANUAL09", "MANUAL_IMG09", "MANUAL10", "MANUAL_IMG10", 
    "MANUAL11", "MANUAL_IMG11", "MANUAL12", "MANUAL_IMG12", "MANUAL13", 
    "MANUAL_IMG13", "MANUAL14", "MANUAL_IMG14", "MANUAL15", "MANUAL_IMG15", 
    "MANUAL16", "MANUAL_IMG16", "MANUAL17", "MANUAL_IMG17", "MANUAL18", 
    "MANUAL_IMG18", "MANUAL19", "MANUAL_IMG19", "MANUAL20", "MANUAL_IMG20", 
    "RCP_NA_TIP"
]
df_raw = df_raw[sorted_columns]
df_raw.head(1)

,RCP_SEQ,RCP_NM,RCP_WAY2,RCP_PAT2,INFO_WGT,INFO_ENG,INFO_CAR,INFO_PRO,INFO_FAT,INFO_NA,...,MANUAL_IMG16,MANUAL17,MANUAL_IMG17,MANUAL18,MANUAL_IMG18,MANUAL19,MANUAL_IMG19,MANUAL20,MANUAL_IMG20,RCP_NA_TIP
0,28,새우 두부 계란찜,찌기,반찬,,220,3,14,17,99,...,,,,,,,,,,"나트륨의 배출을 도와주는 것으로 알려진 칼륨이 풍부한 시금치와 소금, 간장 등의 양..."


In [182]:
sorted_columns = [
    "RCP_SEQ", "RCP_NM", "RCP_WAY2", "RCP_PAT2", 
    "INFO_WGT", "INFO_ENG", "INFO_CAR", "INFO_PRO", "INFO_FAT", "INFO_NA", 
    "HASH_TAG", 
    "ATT_FILE_NO_MAIN", "ATT_FILE_NO_MK", "RCP_PARTS_DTLS", 
    "MANUAL01", "MANUAL_IMG01", "MANUAL02", "MANUAL_IMG02", "MANUAL03", 
    "MANUAL_IMG03", "MANUAL04", "MANUAL_IMG04", "MANUAL05", "MANUAL_IMG05", 
    "MANUAL06", "MANUAL_IMG06", "MANUAL07", "MANUAL_IMG07", "MANUAL08", 
    "MANUAL_IMG08", "MANUAL09", "MANUAL_IMG09", "MANUAL10", "MANUAL_IMG10", 
    "MANUAL11", "MANUAL_IMG11", "MANUAL12", "MANUAL_IMG12", "MANUAL13", 
    "MANUAL_IMG13", "MANUAL14", "MANUAL_IMG14", "MANUAL15", "MANUAL_IMG15", 
    "MANUAL16", "MANUAL_IMG16", "MANUAL17", "MANUAL_IMG17", "MANUAL18", 
    "MANUAL_IMG18", "MANUAL19", "MANUAL_IMG19", "MANUAL20", "MANUAL_IMG20", 
    "RCP_NA_TIP"
]

In [183]:
df_raw.shape

(1136, 55)

- 빈문자열 -> Null 값 처리 (Null 값 처리 후 빈문자열로 원상복귀)

In [184]:
df_raw.replace("", None, inplace=True)
df_raw.dtypes

RCP_SEQ             object
RCP_NM              object
RCP_WAY2            object
RCP_PAT2            object
INFO_WGT            object
INFO_ENG            object
INFO_CAR            object
INFO_PRO            object
INFO_FAT            object
INFO_NA             object
HASH_TAG            object
ATT_FILE_NO_MAIN    object
ATT_FILE_NO_MK      object
RCP_PARTS_DTLS      object
MANUAL01            object
MANUAL_IMG01        object
MANUAL02            object
MANUAL_IMG02        object
MANUAL03            object
MANUAL_IMG03        object
MANUAL04            object
MANUAL_IMG04        object
MANUAL05            object
MANUAL_IMG05        object
MANUAL06            object
MANUAL_IMG06        object
MANUAL07            object
MANUAL_IMG07        object
MANUAL08            object
MANUAL_IMG08        object
MANUAL09            object
MANUAL_IMG09        object
MANUAL10            object
MANUAL_IMG10        object
MANUAL11            object
MANUAL_IMG11        object
MANUAL12            object
M

- Null 값 확인

In [185]:
df_raw.isnull().sum()

RCP_SEQ                0
RCP_NM                 0
RCP_WAY2               0
RCP_PAT2               0
INFO_WGT             871
INFO_ENG               0
INFO_CAR               0
INFO_PRO               0
INFO_FAT               0
INFO_NA                0
HASH_TAG             779
ATT_FILE_NO_MAIN       2
ATT_FILE_NO_MK         1
RCP_PARTS_DTLS         3
MANUAL01               3
MANUAL_IMG01           3
MANUAL02               2
MANUAL_IMG02           2
MANUAL03               1
MANUAL_IMG03           1
MANUAL04              51
MANUAL_IMG04          51
MANUAL05              54
MANUAL_IMG05          54
MANUAL06             108
MANUAL_IMG06         108
MANUAL07            1136
MANUAL_IMG07        1136
MANUAL08            1136
MANUAL_IMG08        1136
MANUAL09            1136
MANUAL_IMG09        1136
MANUAL10            1136
MANUAL_IMG10        1136
MANUAL11            1136
MANUAL_IMG11        1136
MANUAL12            1136
MANUAL_IMG12        1136
MANUAL13            1136
MANUAL_IMG13        1136


- 필요없는 칼럼 제거 (Manual7 ~ Manual 20: 모든 행이 Null 값임)

In [186]:
rlen = len(df_raw)
columns_to_drop = df_raw.columns[df_raw.isnull().sum() == rlen]
df = df_raw.drop(columns=columns_to_drop)
print(columns_to_drop)
df.shape

Index(['MANUAL07', 'MANUAL_IMG07', 'MANUAL08', 'MANUAL_IMG08', 'MANUAL09',
       'MANUAL_IMG09', 'MANUAL10', 'MANUAL_IMG10', 'MANUAL11', 'MANUAL_IMG11',
       'MANUAL12', 'MANUAL_IMG12', 'MANUAL13', 'MANUAL_IMG13', 'MANUAL14',
       'MANUAL_IMG14', 'MANUAL15', 'MANUAL_IMG15', 'MANUAL16', 'MANUAL_IMG16',
       'MANUAL17', 'MANUAL_IMG17', 'MANUAL18', 'MANUAL_IMG18', 'MANUAL19',
       'MANUAL_IMG19', 'MANUAL20', 'MANUAL_IMG20'],
      dtype='object')


(1136, 27)

- Null -> 빈문자열로 원상복귀

In [187]:
df = df.fillna("")

- 데이터 타입 변환

In [188]:
df = df.astype(str)

numeric_columns = ["RCP_SEQ", "INFO_ENG", "INFO_CAR", "INFO_PRO", "INFO_FAT", "INFO_NA"]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")
df.dtypes

RCP_SEQ               int64
RCP_NM               object
RCP_WAY2             object
RCP_PAT2             object
INFO_WGT             object
INFO_ENG            float64
INFO_CAR            float64
INFO_PRO            float64
INFO_FAT            float64
INFO_NA             float64
HASH_TAG             object
ATT_FILE_NO_MAIN     object
ATT_FILE_NO_MK       object
RCP_PARTS_DTLS       object
MANUAL01             object
MANUAL_IMG01         object
MANUAL02             object
MANUAL_IMG02         object
MANUAL03             object
MANUAL_IMG03         object
MANUAL04             object
MANUAL_IMG04         object
MANUAL05             object
MANUAL_IMG05         object
MANUAL06             object
MANUAL_IMG06         object
RCP_NA_TIP           object
dtype: object

In [189]:
df = df.astype(str)

numeric_columns = ["RCP_SEQ", "INFO_ENG", "INFO_CAR", "INFO_PRO", "INFO_FAT", "INFO_NA"]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")
df.dtypes

RCP_SEQ               int64
RCP_NM               object
RCP_WAY2             object
RCP_PAT2             object
INFO_WGT             object
INFO_ENG            float64
INFO_CAR            float64
INFO_PRO            float64
INFO_FAT            float64
INFO_NA             float64
HASH_TAG             object
ATT_FILE_NO_MAIN     object
ATT_FILE_NO_MK       object
RCP_PARTS_DTLS       object
MANUAL01             object
MANUAL_IMG01         object
MANUAL02             object
MANUAL_IMG02         object
MANUAL03             object
MANUAL_IMG03         object
MANUAL04             object
MANUAL_IMG04         object
MANUAL05             object
MANUAL_IMG05         object
MANUAL06             object
MANUAL_IMG06         object
RCP_NA_TIP           object
dtype: object

- `RCP_PARTS_DTLS` 칼럼이 Null 인 경우 날리기

In [190]:
df[df.RCP_PARTS_DTLS == ""]

,RCP_SEQ,RCP_NM,RCP_WAY2,RCP_PAT2,INFO_WGT,INFO_ENG,INFO_CAR,INFO_PRO,INFO_FAT,INFO_NA,...,MANUAL_IMG02,MANUAL03,MANUAL_IMG03,MANUAL04,MANUAL_IMG04,MANUAL05,MANUAL_IMG05,MANUAL06,MANUAL_IMG06,RCP_NA_TIP
493,691,호박잎다슬기된장국,끓이기,국&찌개,,86.7,6.2,9.5,5.9,577.2,...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,3. 멸치다시마국물 1컵에 된장을 풀고 씻은 다슬기살을 함께 넣어 중불에서 5분정도...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,4. 국물낸 다슬기는 건져 달걀물과 함께 섞는다.,http://www.foodsafetykorea.go.kr/uploadimg/coo...,5. 부추는 1cm 가량의 크기로 썰고 대파는 어슷썰기 한다.,http://www.foodsafetykorea.go.kr/uploadimg/coo...,"6. 국물이 끓어 오르면 호박잎과 부추, 파, 다진 마늘을 넣고 마지막으로 달걀물을...",http://www.foodsafetykorea.go.kr/uploadimg/coo...,호박잎은 비벼서 연하게 만든 후 사용한다. - 다슬기를 달걀물에 묻혀 넣으면 달걀이...
494,692,양배추두부찜과 양파케첩소스,찌기,반찬,,165.0,14.1,10.7,7.2,478.8,...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,"3. 당근, 양파, 표고버섯은 곱게 다지고 두부는 으깨 면보에 짠다",http://www.foodsafetykorea.go.kr/uploadimg/coo...,"4. 다진 돼지고기에 파, 다진 마늘, 참기름, 설탕, 소금, 후춧가루를 넣고 양념...",http://www.foodsafetykorea.go.kr/uploadimg/coo...,5. 찐 양배추에 밀가루를 뿌리고 3의 재료를 놓고 돌돌 감아 찜통에 찐다.,http://www.foodsafetykorea.go.kr/uploadimg/coo...,"6. 소스는 다진 양파와 마늘을 먼저 볶아 향을 낸 후 토마토케첩, 간장, 매실액을...",http://www.foodsafetykorea.go.kr/uploadimg/coo...,양배추의 단단한 잎줄기는 부드러운 잎과 열을 받는 시차가 다르고 음식을 할 때도 모...
1029,831,곤약 콩조림,끓이기,반찬,,96.4,10.7,7.5,3.6,210.0,...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,3. 콩은 불리고 마늘은 편으로 썬다.,http://www.foodsafetykorea.go.kr/uploadimg/coo...,"4. 마늘쫑은 3cm 길이로 자르고, 파프리카는 1.5 x 4cm로 썬다.",http://www.foodsafetykorea.go.kr/uploadimg/coo...,5. 냄비에 저나트륨 조림소스 재료를 넣고 곤약을 넣어 약한 불에서 졸인다.,http://www.foodsafetykorea.go.kr/uploadimg/coo...,6. 나머지 재료를 넣어 연하게 졸여낸다,http://www.foodsafetykorea.go.kr/uploadimg/coo...,저나트륨 조림소스는 보통 조림소스보다 간장 양을 1/3정도 줄여서 사용하므로 색을 ...


In [191]:
df = df[df['RCP_PARTS_DTLS'] != ""]
df.shape

(1133, 27)

- RCP_SEQ 기준으로 정렬

In [192]:
df = df.sort_values(by="RCP_SEQ", ascending=True).reset_index(drop=True)
df

,RCP_SEQ,RCP_NM,RCP_WAY2,RCP_PAT2,INFO_WGT,INFO_ENG,INFO_CAR,INFO_PRO,INFO_FAT,INFO_NA,...,MANUAL_IMG02,MANUAL03,MANUAL_IMG03,MANUAL04,MANUAL_IMG04,MANUAL05,MANUAL_IMG05,MANUAL06,MANUAL_IMG06,RCP_NA_TIP
0,18,버섯 두유 소스 볶음,볶기,반찬,,285.0,17.0,9.0,20.0,53.0,...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,3. 볶은 새송이버섯을 소스에 넣고 고루 섞어 가며 조금 더 볶아 접시에 담은 후 ...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,,,,,,,두유와 생크림으로 고소한 맛을 내어 짠맛을 대신해 줘요. 두유는 식사대용으로 먹을 ...
1,19,백일송이버섯 볶음,볶기,반찬,,80.0,10.0,3.0,3.0,11.0,...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,3. 버섯이 살짝 숨이 죽으면 파프리카를 넣어 함께 볶아 접시에 담는다.c,http://www.foodsafetykorea.go.kr/uploadimg/coo...,,,,,,,"파프리카에 함유된 칼륨, 철분, 구리, 아연 성분은 체내 나트륨 수치를 낮추는 효과..."
2,21,땅콩소스 버섯가지나물,끓이기,반찬,55,64.2,3.4,4.3,3.7,64.3,...,http://www.foodsafetykorea.go.kr/uploadimg/202...,3. 가지와 버섯은 끓는 물에 각각 데친 후 찬물에 식힌다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,4. 땅콩소스 재료를 섞어 땅콩소스를 만든다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,"5. 데친 가지와 버섯은 물기를 제거해 그릇에 담고, 땅콩소스를 곁들여 마무리한다.",http://www.foodsafetykorea.go.kr/uploadimg/202...,,,•땅콩과 저염간장으로 만든 소스로 나트륨 섭취를 줄이고 짠맛을 고소한 맛으로 대체했어요.
3,28,새우 두부 계란찜,찌기,반찬,,220.0,3.0,14.0,17.0,99.0,...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,3. 시금치를 잘게 다져 혼합물 그릇(2)에 뿌리고 찜기에 넣고 중간 불에서 10분...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,,,,,,,"나트륨의 배출을 도와주는 것으로 알려진 칼륨이 풍부한 시금치와 소금, 간장 등의 양..."
4,29,부추 콩가루 찜,찌기,반찬,,215.0,20.0,14.0,9.0,240.0,...,http://www.foodsafetykorea.go.kr/uploadimg/202...,"3. 저염간장에 다진 대파, 다진 마늘, 고춧가루, 요리당 , 참기름, 참깨를 섞어...",http://www.foodsafetykorea.go.kr/uploadimg/202...,,,,,,,콩가루로 버무려 감칠맛과 고소한 맛으로 나트륨 사용량을 줄여보세요. 부추는 피를 맑...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,3575,소고기 김말이,굽기,반찬,100,255.8,11.8,16.4,15.9,135.2,...,http://www.foodsafetykorea.go.kr/uploadimg/202...,3. 튀김가루와 물(15ml)을 섞어 반죽을 만든다,http://www.foodsafetykorea.go.kr/uploadimg/202...,4. 손질한 소고기는 후추로 밑간하여 한입 크기로 다지고 김 위에 올려 돌돌 말아 ...,http://www.foodsafetykorea.go.kr/uploadimg/202...,5. 김말이에 반죽을 묻히고 달군 팬에 식용유를 두르고 김말이를 앞뒤로 굽는다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,"6. 접시에 담고 양념장, 깨를 올려 마무리한다.",http://www.foodsafetykorea.go.kr/uploadimg/202...,"•향신 재료인 고춧가루, 깨, 쪽파, 마늘 등을 활용하여 풍미를 높여 나트륨과 당류..."
1129,3576,감태샐러드말이와 버섯말이깐풍,굽기,반찬,100,125.2,22.9,2.0,2.8,120.1,...,http://www.foodsafetykorea.go.kr/uploadimg/202...,3. 라이스페이퍼에 올리고 한입 크기로 싼다. 예열한 팬에 식용유(10g)를 두르고...,http://www.foodsafetykorea.go.kr/uploadimg/202...,"4. 물기를 제거한 양상추, 적채를 라이스페이퍼에 올리고 싼 뒤, 감태 위에 올려 ...",http://www.foodsafetykorea.go.kr/uploadimg/202...,"5. 예열한 팬에 식용유(10g)를 두르고 다진 청고추, 홍고추, 대파, 마늘을 볶...",http://www.foodsafetykorea.go.kr/uploadimg/202...,6. 접시에 담고 샐러드 드레싱과 깐풍 소스를 곁들여 마무리한다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,•칼륨이 풍부한 감태를 사용해서 나트륨 배출을 도왔어요. \n<br>•감칠맛이 풍부...
1130,3577,부추 배무침,기타,반찬,90,32.1,5.6,1.5,0.4,305.1,...,http://www.foodsafetykorea.go.kr/uploadimg/202...,"3. 다진 마늘, 고춧가루, 멸치액젓, 저염간장, 레몬즙, 통깨(2g)를 섞어 양념...",http://www.foodsafetykorea.go.kr/uploadimg/202...,"4. 부추, 양파, 배의 물기를 제거한다.",http://www.foodsafetykorea.go.kr/uploadimg/202...,5. 볼에 담고 양념장을 넣어 버무린다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,6. 접시에 담고 통깨(2g)를 뿌려 마무리한다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,•배를 사용하여 설탕 대신 과일의 단맛을 활용했어요.<br>\n•부추를 사용하여 풍...
1131,3578,열무톳김치,기타,반찬,50,13.7,2.2,1.0,0.1,379.3,...,http://www.foodsafetykorea.go.kr/uploadimg/202...,3. 손질한 열무는 끓은 물에 살짝 데치고 물기를 제거한다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,"4. 마늘, 생강은 잘게 다진다.",http://www.foodsafetykorea.go.kr/uploadimg/202...,"5. 볼에 손질한 열무, 톳, 마늘, 생강을 넣고 고춧가루, 어간장으로 간하여 버무린다.",http://www.foodsafetykorea.go.kr/uploadimg/202...,6. 접시에 담고 통깨를 뿌려 마무리한다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,•나트륨 사용량을 줄이기 위해 열무를 절이지 않고 사용했어요.<br>\n•어간장을 ...


## `RCP_PARTS_DTLS` 칼럼 데이터 클렌징
- `[1인분]`, `1인분 기준<br>\n[주재료]` `•필수 재료 :` 등 불필요한 용어 제거 및 순수 재료 데이터로 정제

In [193]:
df_process = pd.DataFrame(df.RCP_PARTS_DTLS, columns=["RCP_PARTS_DTLS"])

In [194]:
df_process = df_process.apply(
    lambda x: x.replace('청·홍', '청홍') \
                .replace('L.A', 'LA') \
                .replace('올ㄹ브오일', '올리브오일') \
                .replace('머스터드조스', '머스터드소스') \
                .replace('후추적당량[양송이 속양파다진것', '후추적당량, 양송이, 양파다진것') \
                .replace('잣 바질페이소스', '잣, 바질페이소스') \
                .replace('굴림만두 새송이버섯', '굴림만두, 새송이버섯') \
                .replace('두부강된장 참기름', '두부강된장, 참기름') \
                .replace('m 적양배추', ', 적양배추') \
                .replace('l고추장', '고추장') \
    if isinstance(x, str) else x
)

In [195]:
with open("datasets/normalized_pattern.pkl", "rb") as f:
    patterns = pickle.load(f)

In [196]:
def contains_food_info(text, pattern):
    return bool(re.search(pattern, text))

In [197]:
def extract_food_names_from_list(text_list, pattern):
    food_names_list = list()
    for origin in text_list:
        text = origin
        if bool(re.search(pattern[0], text)):
            text = re.sub(pattern[0], pattern[1], text)
        
        items = text.split(',')
        
        food_names = []
        for item in items:
            item = item.strip()
            if item:
                food_names.append(item)
        
        food_str = ", ".join(food_names)
        food_names_list.append(food_str)
    
    return food_names_list

In [198]:
df_process

,RCP_PARTS_DTLS
0,"버섯볶음\n새송이버섯 100g(3개), 올리브유 10g(2작은술)\n두유 소스\n두..."
1,"백일송이버섯볶음\n백일송이버섯 65g(1컵), 노랑 파프리카 5g(5×1cm), 빨..."
2,"•필수 재료 : 가지(15g), 새송이버섯(15g), 표고버섯(10g)\n•땅콩 소..."
3,"새우두부계란찜\n연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g..."
4,"[1인분]조선부추 50g, 날콩가루 7g(1⅓작은술)\n·양념장 : 저염간장 3g(..."
...,...
1128,"1인분 기준<br>\n[주재료] 소고기(치맛살)(100g), 김(6g(3장)), 튀..."
1129,"1인분 기준<br>\n[주재료] 감태 (2g(2장)), 라이스페이퍼(60g), 양상..."
1130,"1인분 기준<br>\n[주재료]부추(30g), 양파(10g), 배(15g)<br>\..."
1131,"1인분 기준<br>\n[주재료] 열무(30g), 톳(10g), 마늘(3g), 생강(..."


In [199]:
for i, pattern in enumerate(patterns):
    df_select = df_process[df_process.RCP_PARTS_DTLS.apply(contains_food_info, pattern=pattern[0])]  # 'RCP_PARTS_DTLS'은 실제 해당 컬럼명으로 바꿔야 합니다.
    df_left = df_process[~df_process.RCP_PARTS_DTLS.apply(contains_food_info, pattern=pattern[0])]  # 음식 정보가 없는 행
    total = len(df_select) + len(df_left)
    print(i + 1, "번째: 선택:" , len(df_select) , ", 남은: " , len(df_left) , ", 총: " , total)
    food_names = extract_food_names_from_list(df_process.RCP_PARTS_DTLS, pattern)
    df_process['RCP_PARTS_DTLS'] = food_names
    print()

1 번째: 선택: 106 , 남은:  1027 , 총:  1133

2 번째: 선택: 64 , 남은:  1069 , 총:  1133

3 번째: 선택: 172 , 남은:  961 , 총:  1133

4 번째: 선택: 264 , 남은:  869 , 총:  1133

5 번째: 선택: 265 , 남은:  868 , 총:  1133

6 번째: 선택: 13 , 남은:  1120 , 총:  1133

7 번째: 선택: 29 , 남은:  1104 , 총:  1133

8 번째: 선택: 12 , 남은:  1121 , 총:  1133

9 번째: 선택: 300 , 남은:  833 , 총:  1133

10 번째: 선택: 1132 , 남은:  1 , 총:  1133



In [200]:
df['RCP_PARTS_DTLS'] = df_process['RCP_PARTS_DTLS']

## 데이터 칼럼명 수정

In [201]:
print(df.columns)

Index(['RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_WGT', 'INFO_ENG',
       'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA', 'HASH_TAG',
       'ATT_FILE_NO_MAIN', 'ATT_FILE_NO_MK', 'RCP_PARTS_DTLS', 'MANUAL01',
       'MANUAL_IMG01', 'MANUAL02', 'MANUAL_IMG02', 'MANUAL03', 'MANUAL_IMG03',
       'MANUAL04', 'MANUAL_IMG04', 'MANUAL05', 'MANUAL_IMG05', 'MANUAL06',
       'MANUAL_IMG06', 'RCP_NA_TIP'],
      dtype='object')


In [202]:
column_mapping = {
    "RCP_SEQ": "id",
    'RCP_NM': "name",
    'RCP_WAY2': "cooking_method",
    'RCP_PAT2': "category",
    'INFO_WGT': "weight",
    'INFO_ENG': "calorie",
    'INFO_CAR': "carbohydrate",
    'INFO_PRO': "protein",
    'INFO_FAT': "fat",
    'INFO_NA': "natrium",
    'HASH_TAG': "main_ingredient",
    'ATT_FILE_NO_MAIN': "main_image_low",
    'ATT_FILE_NO_MK': "main_image",
    'RCP_PARTS_DTLS': "ingredient_detail",
    'MANUAL01': "manual_01",
    'MANUAL_IMG01': "manual_image_01",
    'MANUAL02': "manual_02",
    'MANUAL_IMG02': "manual_image_02",
    'MANUAL03': "manual_03",
    'MANUAL_IMG03': "manual_image_03",
    'MANUAL04': "manual_04",
    'MANUAL_IMG04': "manual_image_04",
    'MANUAL05': "manual_05",
    'MANUAL_IMG05': "manual_image_05",
    'MANUAL06': "manual_06",
    'MANUAL_IMG06': "manual_image_06",
    'RCP_NA_TIP': "recipe_description"
}

df.rename(columns=column_mapping, inplace=True)

In [203]:
df["id"] = range(1, len(df) + 1)

## `recipe_cleaned.csv` 파일 저장

In [204]:
df_food_csv = pd.DataFrame(df)
df_food_csv.head()

,id,name,cooking_method,category,weight,calorie,carbohydrate,protein,fat,natrium,...,manual_image_02,manual_03,manual_image_03,manual_04,manual_image_04,manual_05,manual_image_05,manual_06,manual_image_06,recipe_description
0,1,버섯 두유 소스 볶음,볶기,반찬,,285.0,17.0,9.0,20.0,53.0,...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,3. 볶은 새송이버섯을 소스에 넣고 고루 섞어 가며 조금 더 볶아 접시에 담은 후 ...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,,,,,,,두유와 생크림으로 고소한 맛을 내어 짠맛을 대신해 줘요. 두유는 식사대용으로 먹을 ...
1,2,백일송이버섯 볶음,볶기,반찬,,80.0,10.0,3.0,3.0,11.0,...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,3. 버섯이 살짝 숨이 죽으면 파프리카를 넣어 함께 볶아 접시에 담는다.c,http://www.foodsafetykorea.go.kr/uploadimg/coo...,,,,,,,"파프리카에 함유된 칼륨, 철분, 구리, 아연 성분은 체내 나트륨 수치를 낮추는 효과..."
2,3,땅콩소스 버섯가지나물,끓이기,반찬,55,64.2,3.4,4.3,3.7,64.3,...,http://www.foodsafetykorea.go.kr/uploadimg/202...,3. 가지와 버섯은 끓는 물에 각각 데친 후 찬물에 식힌다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,4. 땅콩소스 재료를 섞어 땅콩소스를 만든다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,"5. 데친 가지와 버섯은 물기를 제거해 그릇에 담고, 땅콩소스를 곁들여 마무리한다.",http://www.foodsafetykorea.go.kr/uploadimg/202...,,,•땅콩과 저염간장으로 만든 소스로 나트륨 섭취를 줄이고 짠맛을 고소한 맛으로 대체했어요.
3,4,새우 두부 계란찜,찌기,반찬,,220.0,3.0,14.0,17.0,99.0,...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,3. 시금치를 잘게 다져 혼합물 그릇(2)에 뿌리고 찜기에 넣고 중간 불에서 10분...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,,,,,,,"나트륨의 배출을 도와주는 것으로 알려진 칼륨이 풍부한 시금치와 소금, 간장 등의 양..."
4,5,부추 콩가루 찜,찌기,반찬,,215.0,20.0,14.0,9.0,240.0,...,http://www.foodsafetykorea.go.kr/uploadimg/202...,"3. 저염간장에 다진 대파, 다진 마늘, 고춧가루, 요리당 , 참기름, 참깨를 섞어...",http://www.foodsafetykorea.go.kr/uploadimg/202...,,,,,,,콩가루로 버무려 감칠맛과 고소한 맛으로 나트륨 사용량을 줄여보세요. 부추는 피를 맑...


In [205]:
df_food_csv = df_food_csv.fillna("")
df_food_csv = df_food_csv.astype(str)
df_food_csv.dtypes

id                    object
name                  object
cooking_method        object
category              object
weight                object
calorie               object
carbohydrate          object
protein               object
fat                   object
natrium               object
main_ingredient       object
main_image_low        object
main_image            object
ingredient_detail     object
manual_01             object
manual_image_01       object
manual_02             object
manual_image_02       object
manual_03             object
manual_image_03       object
manual_04             object
manual_image_04       object
manual_05             object
manual_image_05       object
manual_06             object
manual_image_06       object
recipe_description    object
dtype: object

In [206]:
df_food_csv.to_csv("datasets/ingredients_cleand.csv", index=False, encoding="utf-8")

### 데이터 명세서 및 정제된 데이터 확인
| i | 칼럼명 | 칼럼내용 |
|---|---|---|
| 1 | RCP_SEQ | 일련번호 |
| 2 | RCP_NM | 메뉴명 |
| 3 | RCP_WAY2 | 조리방법 |
| 4 | RCP_PAT2 | 요리종류 |
| 5 | INFO_WGT | 중량(1인분) |
| 6 | INFO_ENG | 열량 |
| 7 | INFO_CAR | 탄수화물 |
| 8 | INFO_PRO | 단백질 |
| 9 | INFO_FAT | 지방 |
| 10 | INFO_NA | 나트륨 |
| 11 | HASH_TAG | 해쉬태그 |
| 12 | ATT_FILE_NO_MAIN | 이미지경로(소) |
| 13 | ATT_FILE_NO_MK | 이미지경로(대) |
| 14 | RCP_PARTS_DTLS | 재료정보 |
| 15 | MANUAL01 | 만드는법_01 |
| 16 | MANUAL_IMG01 | 만드는법_이미지_01 |
| 17-52 | ... | ... |
| 53 | MANUAL20 | 만드는법_20 |
| 54 | MANUAL_IMG20 | 만드는법_이미지_20 |
| 55 | RCP_NA_TIP | 저감 조리법 TIP |

<br>

> Manual7 ~ Manual 20은 1136개 중 전부 Null 값으로 실질적으로 필요없는 칼럼입니다.


In [207]:
print(df_food_csv.columns)

Index(['id', 'name', 'cooking_method', 'category', 'weight', 'calorie',
       'carbohydrate', 'protein', 'fat', 'natrium', 'main_ingredient',
       'main_image_low', 'main_image', 'ingredient_detail', 'manual_01',
       'manual_image_01', 'manual_02', 'manual_image_02', 'manual_03',
       'manual_image_03', 'manual_04', 'manual_image_04', 'manual_05',
       'manual_image_05', 'manual_06', 'manual_image_06',
       'recipe_description'],
      dtype='object')


In [208]:
df_food_csv[['weight', 'calorie', 'carbohydrate', 'protein', 'fat', 'natrium',]]

,weight,calorie,carbohydrate,protein,fat,natrium
0,,285.0,17.0,9.0,20.0,53.0
1,,80.0,10.0,3.0,3.0,11.0
2,55,64.2,3.4,4.3,3.7,64.3
3,,220.0,3.0,14.0,17.0,99.0
4,,215.0,20.0,14.0,9.0,240.0
...,...,...,...,...,...,...
1128,100,255.8,11.8,16.4,15.9,135.2
1129,100,125.2,22.9,2.0,2.8,120.1
1130,90,32.1,5.6,1.5,0.4,305.1
1131,50,13.7,2.2,1.0,0.1,379.3


In [209]:
df_food_csv[['main_ingredient', 'main_image', 'ingredient_detail', 'recipe_description']]

,main_ingredient,main_image,ingredient_detail,recipe_description
0,,http://www.foodsafetykorea.go.kr/uploadimg/coo...,"새송이버섯 100g(3개), 올리브유 10g(2작은술), 두유 20g(1⅓큰술), ...",두유와 생크림으로 고소한 맛을 내어 짠맛을 대신해 줘요. 두유는 식사대용으로 먹을 ...
1,백일송이,http://www.foodsafetykorea.go.kr/uploadimg/coo...,"백일송이버섯 65g(1컵), 노랑 파프리카 5g(5×1cm), 빨강 파프리카 5g(...","파프리카에 함유된 칼륨, 철분, 구리, 아연 성분은 체내 나트륨 수치를 낮추는 효과..."
2,,http://www.foodsafetykorea.go.kr/uploadimg/202...,"가지(15g), 새송이버섯(15g), 표고버섯(10g), 땅콩가루(5g), 다진마늘...",•땅콩과 저염간장으로 만든 소스로 나트륨 섭취를 줄이고 짠맛을 고소한 맛으로 대체했어요.
3,연두부,http://www.foodsafetykorea.go.kr/uploadimg/coo...,"연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g(1/2개), 생...","나트륨의 배출을 도와주는 것으로 알려진 칼륨이 풍부한 시금치와 소금, 간장 등의 양..."
4,날콩가루,http://www.foodsafetykorea.go.kr/uploadimg/coo...,"조선부추 50g, 날콩가루 7g(1⅓작은술), 저염간장 3g(2/3작은술), 다진 ...",콩가루로 버무려 감칠맛과 고소한 맛으로 나트륨 사용량을 줄여보세요. 부추는 피를 맑...
...,...,...,...,...
1128,,http://www.foodsafetykorea.go.kr/uploadimg/202...,"소고기(치맛살)(100g), 김(6g(3장)), 튀김가루(20g), 식용유(45g)...","•향신 재료인 고춧가루, 깨, 쪽파, 마늘 등을 활용하여 풍미를 높여 나트륨과 당류..."
1129,,http://www.foodsafetykorea.go.kr/uploadimg/202...,"감태 (2g(2장)), 라이스페이퍼(60g), 양상추(80g), 적채(20g), 건...",•칼륨이 풍부한 감태를 사용해서 나트륨 배출을 도왔어요. \n<br>•감칠맛이 풍부...
1130,,http://www.foodsafetykorea.go.kr/uploadimg/202...,"부추(30g), 양파(10g), 배(15g), 다진 마늘(8g), 고춧가루(8g),...",•배를 사용하여 설탕 대신 과일의 단맛을 활용했어요.<br>\n•부추를 사용하여 풍...
1131,,http://www.foodsafetykorea.go.kr/uploadimg/202...,"열무(30g), 톳(10g), 마늘(3g), 생강(1g), 어간장(5g), 고춧가루...",•나트륨 사용량을 줄이기 위해 열무를 절이지 않고 사용했어요.<br>\n•어간장을 ...


In [210]:
df_food_csv[['manual_01', 'manual_02', 'manual_03', 'manual_04', 'manual_05', 'manual_06',]]

,manual_01,manual_02,manual_03,manual_04,manual_05,manual_06
0,1. 새송이버섯은 굵게 채썬 후 달군 프라이팬에 기름을 두르고 노릇하게 볶는다.a,"2. 소스 팬에 두유, 생크림, 청양고추, 후추를 넣고 약한 불에 뭉근하게 졸인다.b",3. 볶은 새송이버섯을 소스에 넣고 고루 섞어 가며 조금 더 볶아 접시에 담은 후 ...,,,
1,1. 파프리카는 씨를 빼고 버섯 크기와 비슷하게 썬다.a,2. 달구어진 팬에 카놀라유를 두르고 백일송이 버섯을 볶는다.b,3. 버섯이 살짝 숨이 죽으면 파프리카를 넣어 함께 볶아 접시에 담는다.c,,,
2,1. 가지는 꼭지를 떼어 편으로 썬다.,"2. 새송이버섯은 길게 편으로 썰고, 표고버섯은 기둥을 떼고 납작하게 썬다.",3. 가지와 버섯은 끓는 물에 각각 데친 후 찬물에 식힌다.,4. 땅콩소스 재료를 섞어 땅콩소스를 만든다.,"5. 데친 가지와 버섯은 물기를 제거해 그릇에 담고, 땅콩소스를 곁들여 마무리한다.",
3,1. 손질된 새우를 끓는 물에 데쳐 건진다.a,"2. 연두부, 달걀, 생크림, 설탕에 녹인 무염버터를 믹서에 넣고 간 뒤 새우(1)...",3. 시금치를 잘게 다져 혼합물 그릇(2)에 뿌리고 찜기에 넣고 중간 불에서 10분...,,,
4,"1. 부추는 깨끗이 씻어 물기를 제거하고, 5cm 길이로 썰고 부추에 날콩가루를 넣...",2. 찜기에 면보를 깔고 부추를 넣은 후 김이 오르게 쪄서 파랗게 익힌다.,"3. 저염간장에 다진 대파, 다진 마늘, 고춧가루, 요리당 , 참기름, 참깨를 섞어...",,,
...,...,...,...,...,...,...
1128,1. 소고기는 키친타월로 핏물을 제거한다.,"2. 다진 쪽파, 다진 마늘, 저염간장, 고춧가루, 깨(18g), 물(4ml)을 섞...",3. 튀김가루와 물(15ml)을 섞어 반죽을 만든다,4. 손질한 소고기는 후추로 밑간하여 한입 크기로 다지고 김 위에 올려 돌돌 말아 ...,5. 김말이에 반죽을 묻히고 달군 팬에 식용유를 두르고 김말이를 앞뒤로 굽는다.,"6. 접시에 담고 양념장, 깨를 올려 마무리한다."
1129,"1. 양상추, 적채는 채 썰어 찬물에 담근다. 마요네즈, 홀그레인 머스터드, 파인애...","2. 데친 느타리버섯, 만가닥버섯과 불린 건표고버섯의 물기를 제거하고 깍둑 썰기하여...",3. 라이스페이퍼에 올리고 한입 크기로 싼다. 예열한 팬에 식용유(10g)를 두르고...,"4. 물기를 제거한 양상추, 적채를 라이스페이퍼에 올리고 싼 뒤, 감태 위에 올려 ...","5. 예열한 팬에 식용유(10g)를 두르고 다진 청고추, 홍고추, 대파, 마늘을 볶...",6. 접시에 담고 샐러드 드레싱과 깐풍 소스를 곁들여 마무리한다.
1130,"1. 부추는 다듬어 5~6cm 크기로 썰고, 양파는 뿌리와 껍질을 제거해 얇게 채 썬다.",2. 배는 껍질을 제거해 여덟 등분하고 씨를 제거하여 편 썰기를 한다.,"3. 다진 마늘, 고춧가루, 멸치액젓, 저염간장, 레몬즙, 통깨(2g)를 섞어 양념...","4. 부추, 양파, 배의 물기를 제거한다.",5. 볼에 담고 양념장을 넣어 버무린다.,6. 접시에 담고 통깨(2g)를 뿌려 마무리한다.
1131,1. 톳은 깨끗한 물에 여러 번 씻은 후 끓는 물에 살짝 데치고 물기를 제거한다,2. 열무는 3cm 크기로 썬다.,3. 손질한 열무는 끓은 물에 살짝 데치고 물기를 제거한다.,"4. 마늘, 생강은 잘게 다진다.","5. 볼에 손질한 열무, 톳, 마늘, 생강을 넣고 고춧가루, 어간장으로 간하여 버무린다.",6. 접시에 담고 통깨를 뿌려 마무리한다.


In [211]:
df_food_csv[['manual_image_01', 'manual_image_02', 'manual_image_03', 'manual_image_04', 'manual_image_05', 'manual_image_06',]]

,manual_image_01,manual_image_02,manual_image_03,manual_image_04,manual_image_05,manual_image_06
0,http://www.foodsafetykorea.go.kr/uploadimg/coo...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,,,
1,http://www.foodsafetykorea.go.kr/uploadimg/coo...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,,,
2,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,
3,http://www.foodsafetykorea.go.kr/uploadimg/coo...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,,,
4,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,,,
...,...,...,...,...,...,...
1128,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...
1129,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...
1130,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...
1131,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...,http://www.foodsafetykorea.go.kr/uploadimg/202...
